In [2]:
### GENERATE
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
TSKS33 Load data for hands-on session 5

Erik G. Larsson 2020
"""

import sys
import snap
import numpy
sys.path.append("/courses/TSKS33/ht2023/common-functions")
from save_Gephi_gexf import saveGephi_gexf
from save_Gephi_gexf import saveGephi_gexf_twocolors
from gen_stochblock import gen_stoch_block_2comm



### GENERATE

#G1=gen_stoch_block_2comm(700,300,3.0/700,3.0/300,0.1/(700+300))
G1=gen_stoch_block_2comm(700,300,10.0/700,10.0/300,0.3/(700+300))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-small-network.txt")

G1=gen_stoch_block_2comm(10000,5000,3.0/10000,3.0/5000,0.1/(10000+5000))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-large-network.txt")


### LOAD FROM FILES

G = snap.LoadEdgeList(snap.PUNGraph, "test-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "test-network.mat")

G = snap.LoadEdgeList(snap.PUNGraph, "karate-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "karate-network.mat")

G = snap.LoadEdgeList(snap.PUNGraph, "SB-small-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-small-network.mat")

G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-large-network.mat")

#G1=gen_stoch_block_2comm(700,300,3.0/700,3.0/300,0.1/(700+300))
G1=gen_stoch_block_2comm(700,300,10.0/700,10.0/300,0.3/(700+300))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-small-network.txt")

G1=gen_stoch_block_2comm(10000,5000,3.0/10000,3.0/5000,0.1/(10000+5000))
G=snap.GetMxScc(G1)
snap.SaveEdgeList(G, "SB-large-network.txt")


### LOAD FROM FILES

G = snap.LoadEdgeList(snap.PUNGraph, "test-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "test-network.mat")

#G = snap.LoadEdgeList(snap.PUNGraph, "karate-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "karate-network.mat")

#G = snap.LoadEdgeList(snap.PUNGraph, "SB-small-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-small-network.mat")

#G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)
#snap.SaveMatlabSparseMtx(G, "SB-large-network.mat")


In [3]:
G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()
import scipy.sparse as sp
import numpy as np

def create_sparse_matrix(snapGraph: snap.PUNGraph):
    rows = []
    cols = []
    data = []
    
    for edge in G.Edges():
        rows.append(edge.GetSrcNId()-1)
        cols.append(edge.GetDstNId()-1)
        data.append(1) 
    coo_matrix = sp.coo_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    return coo_matrix



def getA(snapGraph: snap.PUNGraph):
    A = np.zeros((num_nodes, num_nodes))
    for edge in snapGraph.Edges():
        A[edge.GetSrcNId()-1][edge.GetDstNId()-1] = 1
    return A
    
def getZ(sparse_matrix: sp.coo_matrix):
    sub_matrix = np.ones((num_nodes, num_nodes))/(2*num_edges)
    result_matrix = sp.coo_matrix(sparse_matrix.toarray()-sub_matrix)
    return result_matrix

def getK(snapGraph: snap.PUNGraph):
    i = 0
    k = np.zeros(num_nodes)
    for node in snapGraph.Nodes():
        k[i] = node.GetDeg()
        i+=1
    return k
    


In [13]:
import snap_scipy
import scipy.sparse as sp
import numpy as np
G = snap.LoadEdgeList(snap.PUNGraph, "karate-network.txt", 0, 1)
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()

def power_method(A):
    x = np.random.rand(num_nodes, 1)
    for _ in range(250):
        x = (A@x) - ((k@(k.T@x))/(2*num_edges))
        x /= np.linalg.norm(x)
        eig = x.T@((A@x) - ((k@(k.T@x))/(2*num_edges)))
    return x, eig.item()

def find_eigenvector_corresponding_to_largest_eigen_value(A):
    x, lambda_hat = power_method(A)
    print("largest-magnitude eigenvalue: ", lambda_hat)
    if(lambda_hat>0):
        print("largest-positive eigenvalue: ", lambda_hat)
        print("largest-positive eigenvector: \n", x)
        return x, lambda_hat
    else:
        lambda_old = lambda_hat
        new_A = A-np.eye(num_nodes)*lambda_hat
        x, lambda_hat = power_method(new_A)
        print("largest-positive eigenvalue: ", lambda_hat+lambda_old)
        print("largest-positive eigenvector: \n", x)
        return x, lambda_hat



x, lambda_hat = find_eigenvector_corresponding_to_largest_eigen_value(A)


largest-magnitude eigenvalue:  4.336516625025874
largest-positive eigenvalue:  4.336516625025874
largest-positive eigenvector: 
 [[0.00152783]
 [0.0065063 ]
 [0.00165355]
 ...
 [0.00432614]
 [0.00102508]
 [0.0015299 ]]


In [22]:
import snap_scipy
import scipy.sparse as sp
import numpy as np
G = snap.LoadEdgeList(snap.PUNGraph, "test-network.txt", 0, 1)
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()

print("test")

x, lambda_hat = find_eigenvector_corresponding_to_largest_eigen_value(A)

from save_Gephi_gexf import save_csrmatrix_Gephi_gexf_twocolors
save_csrmatrix_Gephi_gexf_twocolors(A, "test.gexf", x)

A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)
num_edges = G.GetEdges()
kkt = np.outer(k, k)/(2*num_edges)
kkt_sparse = sp.csr_matrix(kkt)
Z = A - kkt_sparse

s = np.where(x > 0, 1, -1)

Q = (s.T@Z.toarray()@s)/(4*num_edges)


print("modularity: ", Q)

test
largest-magnitude eigenvalue:  -1.9192868848312765
largest-positive eigenvalue:  1.3549980049551584
largest-positive eigenvector: 
 [[ 0.37200009]
 [ 0.45877098]
 [ 0.37200009]
 [-0.25823163]
 [-0.47226977]
 [-0.47226977]]
modularity:  [[0.31944444]]


In [23]:
import snap_scipy
import scipy.sparse as sp
import numpy as np
G = snap.LoadEdgeList(snap.PUNGraph, "karate-network.txt", 0, 1)
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()

print("karate")

x, lambda_hat = find_eigenvector_corresponding_to_largest_eigen_value(A)

from save_Gephi_gexf import save_csrmatrix_Gephi_gexf_twocolors
save_csrmatrix_Gephi_gexf_twocolors(A, "karate.gexf", x)

A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)
num_edges = G.GetEdges()
kkt = np.outer(k, k)/(2*num_edges)
kkt_sparse = sp.csr_matrix(kkt)
Z = A - kkt_sparse

s = np.where(x > 0, 1, -1)

Q = (s.T@Z.toarray()@s)/(4*num_edges)


print("modularity: ", Q)


karate
largest-magnitude eigenvalue:  -5.592496342798063
largest-positive eigenvalue:  4.977080225729886
largest-positive eigenvector: 
 [[-0.38754293]
 [-0.26955742]
 [-0.13189899]
 [-0.25345236]
 [-0.13400323]
 [-0.14573804]
 [-0.14573804]
 [-0.20935954]
 [ 0.05447571]
 [-0.13400323]
 [-0.07784279]
 [-0.12874396]
 [-0.13502861]
 [-0.13197981]
 [-0.05764888]
 [-0.13197981]
 [ 0.10185678]
 [ 0.09626391]
 [ 0.04785238]
 [ 0.10276487]
 [ 0.06834027]
 [ 0.32390453]
 [-0.05851821]
 [ 0.36983786]
 [ 0.13943288]
 [ 0.13943288]
 [ 0.13943288]
 [ 0.13943288]
 [ 0.13943288]
 [ 0.21674712]
 [ 0.07540039]
 [ 0.20629456]
 [ 0.0563305 ]
 [ 0.11580257]]
modularity:  [[0.37146614]]


In [17]:
import snap_scipy
import scipy.sparse as sp
import numpy as np
G = snap.LoadEdgeList(snap.PUNGraph, "SB-small-network.txt", 0, 1)
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()

print("sb-small")

x, lambda_hat = find_eigenvector_corresponding_to_largest_eigen_value(A)

from save_Gephi_gexf import save_csrmatrix_Gephi_gexf_twocolors
save_csrmatrix_Gephi_gexf_twocolors(A, "sb-small.gexf", x)

A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)
num_edges = G.GetEdges()
kkt = np.outer(k, k)/(2*num_edges)
kkt_sparse = sp.csr_matrix(kkt)
Z = A - kkt_sparse

s = np.where(x > 0, 1, -1)

Q = (s.T@Z.toarray()@s)/(4*num_edges)


print("modularity: ", Q)

sb-small
largest-magnitude eigenvalue:  10.851082330464475
largest-positive eigenvalue:  10.851082330464475
largest-positive eigenvector: 
 [[ 0.0179835 ]
 [ 0.01545068]
 [ 0.02497645]
 [ 0.03701108]
 [ 0.02048997]
 [ 0.02417604]
 [ 0.00921354]
 [ 0.02248944]
 [ 0.02223616]
 [ 0.01759021]
 [ 0.03711477]
 [ 0.02704776]
 [ 0.03564188]
 [ 0.02907046]
 [ 0.02416069]
 [ 0.02887876]
 [ 0.02113265]
 [ 0.02774279]
 [ 0.02651943]
 [ 0.02690071]
 [ 0.01856219]
 [ 0.02214877]
 [ 0.02626707]
 [ 0.01147116]
 [ 0.02783292]
 [ 0.02206924]
 [ 0.02461012]
 [ 0.01862688]
 [ 0.02606725]
 [ 0.00773274]
 [ 0.02335656]
 [ 0.02675299]
 [ 0.02547365]
 [ 0.03817373]
 [ 0.01818891]
 [ 0.0123797 ]
 [ 0.01407971]
 [ 0.01578724]
 [ 0.02790656]
 [ 0.01442993]
 [ 0.02804528]
 [ 0.01299882]
 [ 0.01974611]
 [ 0.01778119]
 [ 0.01474553]
 [ 0.01071548]
 [ 0.03687767]
 [ 0.0123461 ]
 [ 0.02683198]
 [ 0.0152153 ]
 [ 0.02416636]
 [ 0.02035683]
 [ 0.01977385]
 [ 0.03406794]
 [ 0.01480217]
 [ 0.0159468 ]
 [ 0.01605084]
 [ 0.

In [20]:
import snap_scipy
import scipy.sparse as sp
import numpy as np
G = snap.LoadEdgeList(snap.PUNGraph, "SB-large-network.txt", 0, 1)
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)

num_nodes = G.GetNodes()
num_edges = G.GetEdges()

print("sb-large")

x, lambda_hat = find_eigenvector_corresponding_to_largest_eigen_value(A)

from save_Gephi_gexf import save_csrmatrix_Gephi_gexf_twocolors
save_csrmatrix_Gephi_gexf_twocolors(A, "sb-large.gexf", x)

A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)
num_edges = G.GetEdges()
kkt = np.outer(k, k)/(2*num_edges)
kkt_sparse = sp.csr_matrix(kkt)
Z = A - kkt_sparse

s = np.where(x > 0, 1, -1)

Q = (s.T@Z.toarray()@s)/(4*num_edges)

print("modularity: ", Q)

sb-large
largest-magnitude eigenvalue:  4.336516625303209
largest-positive eigenvalue:  4.336516625303209
largest-positive eigenvector: 
 [[0.00152783]
 [0.00650635]
 [0.00165353]
 ...
 [0.00432616]
 [0.00102508]
 [0.00152988]]
modularity:  [[0.43214913]]


In [5]:
from save_Gephi_gexf import save_csrmatrix_Gephi_gexf_twocolors
save_csrmatrix_Gephi_gexf_twocolors(A, "large_sb.gexf", x)


In [15]:
A = snap_scipy.to_sparse_mat(G)
k = np.sum(A, axis=1)
num_edges = G.GetEdges()
kkt = np.outer(k, k)/(2*num_edges)
kkt_sparse = sp.csr_matrix(kkt)
Z = A - kkt_sparse

s = np.where(x > 0, 1, -1)

Q = (s.T@Z.toarray()@s)/(4*num_edges)

print(Q)

KeyboardInterrupt: 

In [7]:
#eigs = np.linalg.eigvals(Z.toarray())

In [8]:
from scipy.sparse.linalg import eigsh
eigenvalues_sparse, _ = eigsh(sp.csr_matrix(Z), k=5, which='LM')

In [9]:
print(eigenvalues_sparse)

[-4.10368488 -4.06403797  4.06914161  4.12107589  4.33651663]
